# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Entrainement d'un modèle Word2Vec sur ce corpus (modèle 10)

In [17]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=2, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 28min 15s, sys: 25.5 s, total: 28min 41s
Wall time: 14min 13s


## Explorer le modèle

In [3]:
model = Word2Vec.load("../data/bulletins_10.model")

### Calculer la similarité entre deux termes

In [4]:
model.wv.similarity("boucher", "boulanger")

0.88451433

In [5]:
model.wv.similarity("homme", "femme")

0.5493822

In [6]:
model.wv.similarity("voiture", "carrosse")

0.3774972

### Chercher les mots les plus proches d'un terme donné

In [7]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8489513397216797),
 ('paris', 0.8009144067764282),
 ('liege', 0.7990047335624695),
 ('nivelles', 0.7926765084266663),
 ('considere_comme_officiel', 0.7591863870620728),
 ('gand', 0.7544216513633728),
 ('rruxelles', 0.7373996376991272),
 ('huy', 0.7307796478271484),
 ('louvain', 0.7209203839302063),
 ('binche', 0.7185885310173035)]

In [8]:
model.wv.most_similar("boucher", topn=10)

[('menuisier', 0.9059860110282898),
 ('batelier', 0.8974393010139465),
 ('marbrier', 0.8947756290435791),
 ('tailleur', 0.8872564435005188),
 ('boulanger', 0.8845143914222717),
 ('cordonnier', 0.8701961040496826),
 ('carrossier', 0.8672388195991516),
 ('commissionnaire', 0.8648064136505127),
 ('cabaretier', 0.8515767455101013),
 ('charcutier', 0.8497314453125)]

### Faire des recherches complexes à travers l'espace vectoriel

In [9]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('huy', 0.8321443200111389), ('paris', 0.8168970346450806), ('manchester', 0.8040940761566162), ('verviers', 0.798650860786438), ('norvege', 0.7958981394767761), ('lille', 0.7932437658309937), ('gand', 0.7927877902984619), ('prague', 0.7915379405021667), ('zurich', 0.7848337292671204), ('tournai', 0.7804679870605469)]


In [10]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('aux_flambeaux', 0.721421480178833), ('communaute_religieuse', 0.7197997570037842), ('federation_bruxelloise', 0.699807345867157), ('vdle', 0.6950361132621765), ('cuur', 0.683005690574646), ('gourde', 0.6818018555641174), ('confederation_nationale', 0.6745078563690186), ('demeurante', 0.6704278588294983), ('representation_dramatique_gratuite_flamande', 0.6599694490432739), ('conference_monetaire', 0.6551566123962402)]


### Modèle 11: Taille du vecteur = 100 (vs 32)

In [24]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=2, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 39min 11s, sys: 36.1 s, total: 39min 47s
Wall time: 20min 7s


In [11]:
model = Word2Vec.load("../data/bulletins_11.model")

In [12]:
model.wv.similarity("boucher", "boulanger")

0.74020714

In [13]:
model.wv.similarity("homme", "femme")

0.47778884

In [55]:
model.wv.similarity("voiture", "carrosse")

0.22808711

In [14]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.7136662602424622),
 ('nivelles', 0.6468287110328674),
 ('prague', 0.6252561807632446),
 ('liege', 0.6158735752105713),
 ('paris', 0.6153978705406189),
 ('bruxelle', 0.6121673583984375),
 ('lille', 0.6068112850189209),
 ('laeken', 0.6047728657722473),
 ('gand', 0.6014081835746765),
 ('lhotel', 0.5876725912094116)]

In [15]:
model.wv.most_similar("boucher", topn=10)

[('cabaretier', 0.7643478512763977),
 ('menuisier', 0.7573539018630981),
 ('cordonnier', 0.7483867406845093),
 ('boulanger', 0.7402071356773376),
 ('charcutier', 0.7394126653671265),
 ('tapissier', 0.7257411479949951),
 ('serrurier', 0.7218273282051086),
 ('marbrier', 0.7206658720970154),
 ('tailleur', 0.7194016575813293),
 ('terrassier', 0.7117941975593567)]

In [16]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('prague', 0.6978469491004944), ('paris', 0.6645860075950623), ('lille', 0.6635715961456299), ('tournai', 0.6521830558776855), ('londres', 0.6501026749610901), ('liege', 0.6492911577224731), ('gand', 0.6386280655860901), ('huy', 0.6197343468666077), ('milan', 0.5927150845527649), ('hambourg', 0.5915898084640503)]


In [17]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('famille_royale', 0.5253315567970276), ('fille', 0.5147914886474609), ('eoi', 0.5106225609779358), ('cujus', 0.5047848224639893), ('saintnicolas', 0.49845537543296814), ('mere', 0.49611812829971313), ('general_belliard', 0.47663527727127075), ('sa_majeste', 0.47655728459358215), ('societe_chorale', 0.4701026678085327), ('legion_nationale', 0.4682620167732239)]


### Modèle 12: Vecteur 100 + élargissement de la fenêtre à 7

In [31]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=7, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 49min 7s, sys: 1min 11s, total: 50min 19s
Wall time: 16min 36s


In [57]:
model = Word2Vec.load("../data/bulletins_12.model")

In [58]:
model.wv.similarity("boucher", "boulanger")

0.74992484

In [59]:
model.wv.similarity("homme", "femme")

0.49970785

In [60]:
model.wv.similarity("voiture", "carrosse")

0.2508179

In [21]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.6743203401565552),
 ('liege', 0.6394780874252319),
 ('prague', 0.6360003352165222),
 ('bruxelle', 0.6343247890472412),
 ('xelles', 0.6240631341934204),
 ('bruxe', 0.6227555871009827),
 ('nivelles', 0.6150801181793213),
 ('lille', 0.612572193145752),
 ('paris', 0.6059279441833496),
 ('bruxeles', 0.5985421538352966)]

In [22]:
model.wv.most_similar("boucher", topn=10)

[('charcutier', 0.7554579973220825),
 ('boulanger', 0.7499249577522278),
 ('menuisier', 0.7398019433021545),
 ('cabaretier', 0.7291788458824158),
 ('tailleur', 0.7144469618797302),
 ('batelier', 0.7124482989311218),
 ('serrurier', 0.7025189399719238),
 ('cordonnier', 0.7000163197517395),
 ('coiffeur', 0.698936402797699),
 ('chapelier', 0.697650671005249)]

In [23]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('prague', 0.6655336022377014), ('londres', 0.6648284792900085), ('paris', 0.6592709422111511), ('lille', 0.658177375793457), ('liege', 0.6438037157058716), ('strasbourg', 0.6386304497718811), ('huy', 0.6232372522354126), ('verviers', 0.6213777661323547), ('gand', 0.6208687424659729), ('tournai', 0.610344409942627)]


In [24]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('fille', 0.5155900120735168), ('iication', 0.5074321627616882), ('deput', 0.5027685165405273), ('eoi', 0.5025864243507385), ('louve', 0.49552175402641296), ('gouvernement_provincial', 0.48193249106407166), ('cure', 0.46933111548423767), ('brabanl', 0.46789273619651794), ('general_belliard', 0.4666241407394409), ('princesse_marie', 0.46532654762268066)]


### Modèle 13: Taille fenêtre 20 (vs 5) et vecteur 32 (vs 100)

In [40]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 53min 1s, sys: 1min 24s, total: 54min 26s
Wall time: 17min 25s


In [61]:
model = Word2Vec.load("../data/bulletins_13.model")

In [62]:
model.wv.similarity("boucher", "boulanger")

0.6937786

In [63]:
model.wv.similarity("homme", "femme")

0.49711436

In [64]:
model.wv.similarity("voiture", "carrosse")

0.30251795

In [28]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.651600182056427),
 ('nivelles', 0.638759195804596),
 ('bruxelle', 0.6351412534713745),
 ('liege', 0.6297926902770996),
 ('xelles', 0.6212089657783508),
 ('bruxe', 0.6058027148246765),
 ('bruxeles', 0.5974170565605164),
 ('thotel', 0.5858594179153442),
 ('paris', 0.5826448798179626),
 ('laeken', 0.5800283551216125)]

In [29]:
model.wv.most_similar("boucher", topn=10)

[('menuisier', 0.6990550756454468),
 ('boulanger', 0.6937785744667053),
 ('cabaretier', 0.6871023774147034),
 ('serrurier', 0.6829543709754944),
 ('batelier', 0.6759117841720581),
 ('tailleur', 0.6719610691070557),
 ('ebeniste', 0.6639692783355713),
 ('plombier', 0.6599940061569214),
 ('charcutier', 0.6555059552192688),
 ('coiffeur', 0.64664626121521)]

In [30]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.6205630302429199), ('paris', 0.6186079382896423), ('lille', 0.617577850818634), ('verviers', 0.6161673069000244), ('londres', 0.6111919283866882), ('huy', 0.6109883189201355), ('tournai', 0.602641224861145), ('gand', 0.596588671207428), ('prague', 0.595768392086029), ('marseille', 0.5875338912010193)]


In [31]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('fille', 0.4998374283313751), ('mere', 0.49119219183921814), ('famille_royale', 0.48008471727371216), ('defunt', 0.4772540330886841), ('sa_majeste', 0.465778112411499), ('depouille_mortelle', 0.4591931402683258), ('congregation', 0.4549408555030823), ('prince_umberto', 0.4521107077598572), ('leur_mere', 0.4515117406845093), ('museliere_sera', 0.44228291511535645)]


### Modèle 14 : Vecteur 100 et fenêtre 13

In [50]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=13, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 57min 44s, sys: 1min 12s, total: 58min 57s
Wall time: 18min 45s


In [32]:
model = Word2Vec.load("../data/bulletins_14.model")

In [33]:
model.wv.similarity("boucher", "boulanger")

0.6723962

In [34]:
model.wv.similarity("homme", "femme")

0.49160355

In [35]:
model.wv.similarity("voiture", "carrosse")

0.31495377

In [36]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6420583724975586),
 ('bruxelle', 0.6356337070465088),
 ('manufacturiere', 0.6315798759460449),
 ('nivelles', 0.6286843419075012),
 ('liege', 0.6279160976409912),
 ('dette_active', 0.6258662939071655),
 ('rruxelles', 0.620442807674408),
 ('tournai', 0.584047257900238),
 ('bruxe', 0.5773397088050842),
 ('prague', 0.5762890577316284)]

In [37]:
model.wv.most_similar("boucher", topn=10)

[('charcutier', 0.7419087290763855),
 ('batelier', 0.7248253226280212),
 ('cabaretier', 0.7033025622367859),
 ('marchand', 0.6830555200576782),
 ('serrurier', 0.6813036799430847),
 ('menuisier', 0.6732695698738098),
 ('boulanger', 0.672396183013916),
 ('abatteur', 0.6655840277671814),
 ('meunier', 0.6591508984565735),
 ('tailleur', 0.6587807536125183)]

In [38]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('londres', 0.627091646194458), ('prague', 0.6158853769302368), ('liege', 0.6125531792640686), ('paris', 0.6043033599853516), ('manchester', 0.602205216884613), ('lille', 0.5956189632415771), ('ostende', 0.5921670794487), ('tournai', 0.5895686149597168), ('huy', 0.588439404964447), ('verviers', 0.5736773014068604)]


In [39]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('mere', 0.5091543197631836), ('fille', 0.49800336360931396), ('princesse_marie', 0.48935967683792114), ('deput', 0.47794777154922485), ('cure', 0.46977701783180237), ('casteau', 0.4649643898010254), ('prince_umberto', 0.4642382860183716), ('sieur_van_eesbeeck', 0.4600988030433655), ('zeepaard', 0.45792263746261597), ('gouvernement_provincial', 0.4477558135986328)]


### Modèle 15 : 300 dimensions et élargissement de la fenêtre à 13 et entrainement ++

In [63]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=13, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 1h 50min 13s, sys: 4min 58s, total: 1h 55min 11s
Wall time: 32min 57s


In [40]:
model = Word2Vec.load("../data/bulletins_15.model")

In [41]:
model.wv.similarity("boucher", "boulanger")

0.5587284

In [42]:
model.wv.similarity("homme", "femme")

0.39769208

In [43]:
model.wv.similarity("voiture", "carrosse")

0.26498124

In [44]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.5277640223503113),
 ('dette_active', 0.49114567041397095),
 ('liege', 0.48792389035224915),
 ('bruxelle', 0.4822578728199005),
 ('manufacturiere', 0.4812834858894348),
 ('nivelles', 0.4720197021961212),
 ('xelles', 0.4664071798324585),
 ('binche', 0.4439750015735626),
 ('verviers', 0.44394662976264954),
 ('tournai', 0.4434230625629425)]

In [45]:
model.wv.most_similar("boucher", topn=10)

[('charcutier', 0.5835238695144653),
 ('cordonnier', 0.5590786933898926),
 ('boulanger', 0.5587283372879028),
 ('menuisier', 0.5515034794807434),
 ('abatteur', 0.551483154296875),
 ('tailleur', 0.5508565306663513),
 ('cabaretier', 0.5470849871635437),
 ('serrurier', 0.5364952683448792),
 ('coiffeur', 0.5345790386199951),
 ('marchand', 0.5307872891426086)]

In [46]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('londres', 0.48951879143714905), ('liege', 0.4573286175727844), ('manchester', 0.4476049840450287), ('milan', 0.4397909343242645), ('ostende', 0.4391721785068512), ('prague', 0.4373452961444855), ('marseille', 0.4363729953765869), ('paris', 0.4332381784915924), ('verviers', 0.4286613166332245), ('lille', 0.42616918683052063)]


In [72]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('souverain', 0.4747852385044098), ('prince', 0.45544907450675964), ('citoyen', 0.4522494375705719), ('magistrat', 0.44988343119621277), ('peuple', 0.44894155859947205), ('congres_national', 0.43645983934402466), ('peuple_belge', 0.43318602442741394), ('geste', 0.43007296323776245), ('soldat', 0.42618703842163086), ('gouvernement', 0.4182727634906769)]


### Modèle 16 : Vecteur 300, fenêtre 10

In [79]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 2h 5min 22s, sys: 6min 26s, total: 2h 11min 48s
Wall time: 38min 53s


In [47]:
model = Word2Vec.load("../data/bulletins_16.model")

In [48]:
model.wv.similarity("boucher", "boulanger")

0.594906

In [49]:
model.wv.similarity("homme", "femme")

0.41035208

In [50]:
model.wv.similarity("voiture", "carrosse")

0.22808711

In [51]:
model.wv.most_similar("bruxelles", topn=10)

[('nivelles', 0.5170215368270874),
 ('rruxelles', 0.5137457251548767),
 ('liege', 0.509574294090271),
 ('manufacturiere', 0.4940529763698578),
 ('tournai', 0.47900691628456116),
 ('prague', 0.47698986530303955),
 ('liruxelles', 0.47526562213897705),
 ('lille', 0.4690064787864685),
 ('xelles', 0.46365970373153687),
 ('huy', 0.4627675414085388)]

In [52]:
model.wv.most_similar("boucher", topn=10)

[('cabaretier', 0.611894428730011),
 ('charcutier', 0.6005944609642029),
 ('boulanger', 0.5949059128761292),
 ('serrurier', 0.58980792760849),
 ('marchand', 0.5820729732513428),
 ('tapissier', 0.5787808299064636),
 ('menuisier', 0.5779645442962646),
 ('tailleur', 0.576256275177002),
 ('coiffeur', 0.5692282319068909),
 ('camionneur', 0.567164957523346)]

In [53]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.4911900460720062), ('liege', 0.4882427453994751), ('londres', 0.4805184006690979), ('hambourg', 0.4721403121948242), ('huy', 0.46542081236839294), ('ostende', 0.46507900953292847), ('prague', 0.4622458219528198), ('tournai', 0.45100125670433044), ('nivelles', 0.446857213973999), ('verviers', 0.4414343535900116)]


In [54]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('fille', 0.4048213064670563), ('mere', 0.40387463569641113), ('boi', 0.3755878806114197), ('gouvernement_italien', 0.374500036239624), ('famille_royale', 0.37325236201286316), ('eoi', 0.36761078238487244), ('klary', 0.36225759983062744), ('legation_belge', 0.3591693937778473), ('sa_majeste', 0.3579140305519104), ('koi', 0.351666659116745)]
